In [2]:
import pandas as pd
import numpy as np
import requests as re
from bs4 import BeautifulSoup

In [3]:
def clean_name_col(df, cols, string):
    df = df[cols]
    df = df[df[cols[1]].notna()]
    df['Player'] = df[cols[1]].map(lambda x: x.split()[0] +' ' + x.split()[2] + ' ' + string[:4] if len(x.split()) > 2 else x)
    if string[:4] in ['2017', '2016', '2015', '2014']:
        df['Player'] = df[cols[1]].map(lambda x: x.split()[0] +' ' + x.split()[1] + ' ' + string[:4] if len(x.split()) > 2 else x)
    df['Player'] = df['Player'].map(lambda x: ''.join(i for i in x if i.isalnum() or i == ' '))
    df.drop(columns=[cols[1]], inplace=True)
    return df


def get_preseason_fantasy_rankings(url_strings):
    yearly_list = []
    for string in url_strings:
        r = re.get(f'https://web.archive.org/web/{string}/https://www.fantasypros.com/nfl/rankings/ppr-cheatsheets.php')
        soup = BeautifulSoup(r.content, 'html.parser')
        table = soup.find_all('table')[0] 
        if string[:4] in ['2014', '2015']:
            table = soup.find_all('table')[1]
        df = pd.read_html(str(table))[0]         
        
        if string[:4] == '2014':
            cols = ['Rank', 'Player (team/bye)', 'Best', 'Worst', 'Ave', 'Std Dev', 'ADP']
            df = clean_name_col(df, cols, string)
        elif string[:4] == '2015':
            cols = ['Rank', 'Player (team, bye)', 'Best', 'Worst', 'Avg', 'Std Dev', 'ADP']
            df = clean_name_col(df, cols, string)
        elif string[:4] in ['2016', '2017']:
            cols = ['Rank', 'Player (Team)', 'Best', 'Worst', 'Avg', 'Std Dev', 'ADP']
            df = clean_name_col(df, cols, string)            
        else:
            cols = ['Rank', 'Overall (Team)', 'Best', 'Worst', 'Avg', 'Std Dev', 'ADP']
            df = clean_name_col(df, cols, string)
        yearly_list.append(df)
    df = pd.concat(yearly_list)
    df['Avg'] = df['Avg'].fillna('test')
    df['Avg'] = df.apply(lambda x: x['Ave'] if x['Avg'] == 'test' else x['Avg'], axis=1)
    df.rename(columns={'Avg':'Avg_Preseason_Rank'}, inplace=True)
    df.drop(columns='Ave', inplace=True)
    return df

In [4]:
url_strings = ['20190903180912', '20180903165223', '20170904024425', '20160902165621', '20150901081121', '20140903025545']
df = get_preseason_fantasy_rankings(url_strings)

In [5]:
df.head()

,Rank,Best,Worst,Avg_Preseason_Rank,Std Dev,ADP,Player
1,1,1,5,1.6,0.9,1.0,Saquon Barkley 2019
2,2,1,8,2.3,1.3,2.0,Christian McCaffrey 2019
3,3,1,13,3.0,1.4,3.0,Alvin Kamara 2019
4,4,4,12,6.0,1.4,5.0,DeAndre Hopkins 2019
6,5,4,22,6.3,3.0,8.0,Davante Adams 2019


In [6]:
#drop unecessary columns
df = df.drop(columns=['Rank', 'Best', 'Worst', 'Std Dev', 'ADP'])

In [7]:
df[df['Player'].str.contains("LeVeon Bell")] #checking a player who originally had an apostrophe in their name

,Avg_Preseason_Rank,Player
16,14.0,LeVeon Bell 2019
2,2.4,LeVeon Bell 2018
2,1.8,LeVeon Bell 2017
18,18.5,LeVeon Bell 2016
0,2.6,LeVeon Bell 2015
20,24.3,LeVeon Bell 2014


In [11]:
#There are two mike davis' so we need to remove the second and irrelevant one.
df = df.drop_duplicates(subset='Player')
df[df['Player']=='Mike Davis 2019']

,Avg_Preseason_Rank,Player
179,177.8,Mike Davis 2019


In [12]:
df.to_csv('preseason_rankings.csv', index=False)